# Import

In [1]:
import os
import tensorflow.compat.v1 as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pathlib import Path
import argparse
# %matplotlib inline
plt.rcParams['savefig.dpi'] = 159.1 #图片像素
plt.rcParams['figure.dpi'] = 159.1  #分辨率:512*512

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# Definition of the Architecture

In [2]:
# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 1000
n_encoder_h_2 = 500
n_encoder_h_3 = 250

# 3 hidden layers for decoder
n_decoder_h_1 = 250
n_decoder_h_2 = 500
n_decoder_h_3 = 1000

# Parameters
learning_rate = 0.1
training_epochs = 500
batch_size = 64
display_step = 10

# Batch Normalization 

In [3]:
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

# Definition of the Layer 

In [4]:
def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape of the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Variable, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))

# Definition of the Encoder Part

In [5]:
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Variable, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [262144, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_encoder_h_2, n_encoder_h_3], [n_encoder_h_3], phase_train)

        with tf.variable_scope("code"):
            output = layer(h_3, [n_encoder_h_3, n_code], [n_code], phase_train)

    return output

# Definition of the Decoder Part

In [6]:
def decoder(x, n_code, phase_train):
    """
    Defines the network decoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Variable, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_decoder_h_2, n_decoder_h_3], [n_decoder_h_3], phase_train)

        with tf.variable_scope("output"):
            output = layer(h_3, [n_decoder_h_3, 262144], [262144], phase_train)

    return output

# Definition of the Loss

In [7]:
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op

# Training Function

In [8]:
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

# Evaluation Function

In [9]:
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op: reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        in_image_op = image_summary("input_image", x)
        
        out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        return val_loss, in_image_op, out_image_op, val_summary_op

# Image Summary

In [10]:
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 512, 512, 1])
    return tf.summary.image(label, tensor_reshaped)

# Plotting samples

In [11]:
def plot_sample(samples, size1, size2):
    
    fig1 = plt.figure(figsize=(size1, size2))
    gs = gridspec.GridSpec(size1, size2)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(512, 512), cmap='gray')
        # plt.imshow(sample.reshape(28, 28), cmap='gray')

    return fig1

# Faciliate the path defining process

In [12]:
# Though it's not possible to get the path to the notebook by __file__, os.path is still very useful in dealing with paths and files
# In this case, we can use an alternative: pathlib.Path
"""
code_dir   = os.path.dirname(__file__)
"""
#get the current path of our code
code_dir = Path().resolve()
#create output_dir within the same path
output_dir = os.path.join(code_dir, 'Problem1_b_3/')
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

# Use all images with random shuffling for training the Autoencoder

In [13]:
def next_batch(data, num):
    '''
    Return a total of `num` random samples 
    '''
    #print(len(data))
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = np.array([data[i] for i in idx])

    return data_shuffle

# Read image file from given path

In [14]:
sizePixel1 = 512
sizePixel2 = sizePixel1*sizePixel1
size_d = 32

def read_tensor_from_image_file(path, input_height=sizePixel1, input_width=sizePixel1, input_mean=0, input_std=255):
    
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(path, input_name)
    image_reader = tf.image.decode_png(file_reader, channels = 1)
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0);
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)
    return result 


nSeries = 222
numInSeries = 1
nImages = nSeries*numInSeries #222
img  = np.zeros((nImages, sizePixel1*sizePixel1))
counter = 0

#get the current path of our code
folder = Path().resolve()

for j in range(0,numInSeries):
    for i in range(1,nSeries+1):
        # print(counter,i,j)
        fname = str(i) + '_' + str(j) + '.png'
        path = './data/' + fname
        orig_img = read_tensor_from_image_file(path)
        # vectorize
        img[counter] = orig_img.reshape(-1)
        
        # original size
        # img[counter] = orig_img.reshape(sizePixel1,sizePixel1)
        counter = counter+1

print('Input image shape is:', img.shape)

Input image shape is: (222, 262144)


# Main Function

In [15]:
if __name__ == '__main__':

    #if a python file, please use the 4 lines bellow and comment the "n_code = '1'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '2'
    
    #feel free to change with your own 
    #log_files_path = './autoencoder_logs/'

    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 262144]) # MNIST data image of shape 512*512=262144
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path + 'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path + 'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)

            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                total_batch = int(len(img)/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    # minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)
                    minibatch_x = next_batch(img, batch_size)
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

#                     validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: mnist.validation.images, phase_train: False})
#                     val_writer.add_summary(in_image, sess.run(global_step))
#                     val_writer.add_summary(out_image, sess.run(global_step))
#                     val_writer.add_summary(val_summary, sess.run(global_step))
#                     print("Validation Loss:", validation_loss)

                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, log_files_path + 'mnist_autoencoder_hidden_' + n_code + '_logs/model-checkpoint-' + '%04d' % (epoch+1), global_step=global_step)


            print("Optimization Finished!")
            
            # generate new image sets
            out_image = sess.run([output], feed_dict={x: img, phase_train: False})
            # extract np.array from the list
            out_image = out_image[0]
            
            for k in range(1, len(img)+1):
                # size1 = 5
                # size2 = 5
                # fig1 = plot_sample(out_image[k-1], size1, size2)
                # plt.savefig(output_dir + str(k) + '_1.png', bbox_inches='tight')
                # plt.close(fig1)
                plt.figure("Image", frameon=False)  # figure window name
                plt.imshow(out_image[k-1].reshape(512, 512), cmap='gray')
                plt.axis('off')
                plt.savefig(output_dir + str(k) + '_3.png', bbox_inches='tight')
                plt.close()
                
            print("Generate New Image Sets Done!")
                
            test_loss = sess.run(eval_op, feed_dict={x: img, phase_train: False})
            
            print()
            print("Test Loss:", test_loss)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch: 0001 cost = 270.729563395
Epoch: 0011 cost = 238.635854085
Epoch: 0021 cost = 236.510172526
Epoch: 0031 cost = 232.990046183
Epoch: 0041 cost = 235.931452433
Epoch: 0051 cost = 230.412531535
Epoch: 0061 cost = 230.897705078
Epoch: 0071 cost = 229.570317586
Epoch: 0081 cost = 230.620315552
Epoch: 0091 cost = 227.891342163
Epoch: 0101 cost = 227.049362183
Epoch: 0111 cost = 225.132171631
Epoch: 0121 cost = 225.946517944
Epoch: 0131 cost = 226.123708089
Epoch: 0141 cost = 224.338582357
Epoch: 0151 cost = 224.351801554
Epoch: 0161 cost = 224.620773315
Epoch: 0171 cost = 222.132720947
Epoch: 0181 cost = 219.866027832
Epoch: 0191 cost = 222.473495483
Optimization Finished!
Generate New Image Sets Done!

Test Loss: 220.74165


In [16]:
# plt.rcParams['savefig.dpi'] = 159.1 #图片像素
# plt.rcParams['figure.dpi'] = 159.1  #分辨率
# plt.figure("Image", frameon=False)  # figure window name
# plt.imshow(out_image[52].reshape(512, 512), cmap='gray')
# plt.axis('off')
# plt.savefig(output_dir + 'TEST_1.png', bbox_inches='tight')
# plt.close()